In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
!pip install pyserini
!pip install faiss

!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu
import faiss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.5 MB 26 kB/s 
     |████████████████████████████████| 4.7 MB 45.3 MB/s 
     |████████████████████████████████| 1.2 MB 56.9 MB/s 
     |████████████████████████████████| 1.3 MB 43.5 MB/s 
     |████████████████████████████████| 4.9 MB 35.0 MB/s 
     |████████████████████████████████| 2.0 MB 34.1 MB/s 
     |████████████████████████████████| 13.5 MB 34.0 MB/s 
     |████████████████████████████████| 188 kB 36.0 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 596 kB 53.2 MB/s 
     |████████████████████████████████| 101 kB 6.2 MB/s 
     |████████████████████████████████| 6.6 MB 41.3 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  

In [ ]:
import pandas as pd
import regex as re
import csv
from itertools import islice
import pickle
import numpy as np
import json
import os
import sys
import argparse
from pathlib import Path
from sklearn.model_selection import train_test_split
from pyserini.search import SimpleSearcher

In [ ]:
%cd /gdrive/MyDrive/nlp-yuan_code/FinBERT-QA
from src.process_data import *

/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
RO_FiQA_DATA_PATH = '/gdrive/MyDrive/nlp-data/nlp-qa-datasets/FiQA/FiQA_train_task2/'
RO_FIQA_INDEX = "/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/retriever/lucene-index-fiqa/"
WR_PICKLE_DATA = '/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/'
WR_PICKLE_TRANSIENT_DATA = '/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/transient/'
WR_INTERIM_DATA = '/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/interim/'

In [ ]:
# Document id and Answer text
collection = load_answers_to_df(RO_FiQA_DATA_PATH+"FiQA_train_doc_final.tsv")
# Question id and Question text
queries = load_questions_to_df(RO_FiQA_DATA_PATH+"FiQA_train_question_final.tsv")
# Question id and Answer id pair
qid_docid = load_qid_docid_to_df(RO_FiQA_DATA_PATH+"FiQA_train_question_doc_final.tsv")

In [ ]:
def split_label(qid_docid):
    """
    Split question answer pairs into train, test, validation sets.

    Returns:
        train_label: Dictonary
            key - question id
            value - list of relevant docids
        test_label: Dictonary
            key - question id
            value - list of relevant docids
        valid_label: Dictonary
            key - question id
            value - list of relevant docids
    ----------
    Arguments:
        qid_docid: Dataframe containing the question id and relevant docids
    """
    # Group the answers for each question into a list
    qid_docid = qid_docid.groupby(['qid']).agg(lambda x: tuple(x)).applymap(list).reset_index()
    # Split data
    train_set, test_set = train_test_split(qid_docid, test_size=0.05)
    #train_set, valid_set = train_test_split(train, test_size=0.1)
    # Expand the list of docids into individual rows to represent a single sample
    train_data = train_set.explode('docid')
    test_data = test_set.explode('docid')
    #valid_data = valid_set.explode('docid')

    # Convert data into dictionary - key: qid, value: list of relevant docid
    train_label = label_to_dict(train_data)
    test_label = label_to_dict(test_data)
    #valid_label = label_to_dict(valid_data)

    return train_label, test_label

def split_question(train_label, test_label, queries):
    """
    Split questions into train, test, validation sets.

    Returns:
        train_questions: Dataframe with qids
        test_questions: Dataframe with qids
        valid_questions: Dataframe with qids
    ----------
    Arguments:
        train_label: Dictionary contraining qid and list of relevant docid
        test_label: Dictionary contraining qid and list of relevant docid
        valid_label: Dictionary contraining qid and list of relevant docid
        queries: Dataframe containing the question id and question text
    """
    # Get a list of question ids
    train_q = list(train_label.keys())
    test_q = list(test_label.keys())

    # Split question dataframe into train, test, valid set
    train_questions = queries[queries['qid'].isin(train_q)]
    test_questions = queries[queries['qid'].isin(test_q)]

    return train_questions, test_questions

def create_dataset(question_df, labels, cands_size):
    """Retrieves the top-k candidate answers for a question and
    creates a list of lists of the dataset containing the question id,
    list of relevant answer ids, and the list of answer candidates

    Returns:
        dataset: list of list in the form [qid, [pos ans], [ans candidates]]
    ----------
    Arguments:
        question_df: Dataframe containing the qid and question text
        labels: Dictonary containing the qid to text map
        cands_size: int - number of candidates to retrieve
    """
    dataset = []
    # Calls retriever
    searcher = SimpleSearcher(RO_FIQA_INDEX)
    # For each question
    for i, row in question_df.iterrows():
        qid = row['qid']
        tmp = []
        # Append qid
        tmp.append(qid)
        # Append list of relevant docs
        tmp.append(labels[qid])
        # Retrieves answer candidates
        cands = []
        cands_score = []
        query = row['question']
        query = re.sub('[£€§]', '', query)
        hits = searcher.search(query, k=cands_size)

        for docid in range(0, len(hits)):
            cands.append(int(hits[docid].docid))
            cands_score.append(hits[docid].score)
        # Append candidate answers
        tmp.append(cands)
        tmp.append(cands_score)
        dataset.append(tmp)

    return dataset

def get_dataset(query_path, labels_path, cands_size):
    """Splits the dataset into train, validation, and test set and creates
    the dataset form for training, validation, and testing.

    Returns:
        train_set: list of list in the form [qid, [pos ans], [ans candidates]]
        valid_set: list of list in the form [qid, [pos ans], [ans candidates]]
        test_set: list of list in the form [qid, [pos ans], [ans candidates]]
    ----------
    Arguments:
        query_path: str - path containing a list of qid and questions
        labels_path: str - path containing a list of qid and relevant docid
        cands_size: int - number of candidates to retrieve
    """
    # Question id and Question text
    queries = load_questions_to_df(query_path)
    # Question id and Answer id pair
    qid_docid = load_qid_docid_to_df(labels_path)
    # qid to docid label map
    labels = label_to_dict(qid_docid)
    train_label, test_label = split_label(qid_docid)
    # Split Questions
    train_questions, test_questions = split_question(train_label, test_label, queries)

    print("\nGenerating training set...\n")
    train_set = create_dataset(train_questions, labels, cands_size)
    print("Generating test set...\n")
    test_set = create_dataset(test_questions, labels, cands_size)

    return train_set, test_set

In [ ]:
query_path = RO_FiQA_DATA_PATH + "FiQA_train_question_final.tsv"
labels_path = RO_FiQA_DATA_PATH + "FiQA_train_question_doc_final.tsv"
train_set, test_set = get_dataset(query_path, labels_path, 50)


Generating training set...

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead
Generating test set...

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [ ]:
empty_docs, empty_id = get_empty_docs(collection)
# Remove empty answers from collection of answers
collection_cleaned = collection.drop(empty_id)
# Remove empty answers from qa pairs
qid_docid = qid_docid[~qid_docid['docid'].isin(empty_docs)]

print("Number of answers after cleaning: {}".format(len(collection_cleaned)))
print("Number of QA pairs after cleaning: {}".format(len(qid_docid)))

processed_answers = process_answers(collection_cleaned)
processed_questions = process_questions(queries)

word2index, word2count = create_vocab(processed_answers, processed_questions)

print("Vocab size: {}".format(len(word2index)))
print("Top {} common words: {}".format(35, Counter(word2count).most_common(35)))

qid_to_text, docid_to_text = id_to_text(collection, queries)
qid_to_tokenized_text, docid_to_tokenized_text = id_to_tokenized_text(processed_answers, processed_questions)
text_to_docid = dict([ (docid_to_text[k], k) for k in docid_to_text])

Number of answers after cleaning: 57600
Number of QA pairs after cleaning: 17072
Vocab size: 85034
Top 35 common words: [('the', 371203), ('to', 233559), ('a', 201620), ('you', 166702), ('and', 163066), ('of', 157574), ('is', 129894), ('in', 120019), ('that', 111416), ('for', 89366), ('it', 83822), ('i', 74100), ('your', 68153), ('are', 67255), ('if', 60689), ('be', 59266), ('on', 58382), ('have', 55754), ('as', 50088), ('this', 49868), ('not', 49227), ('or', 46080), ('with', 45894), ('they', 44485), ('but', 41690), ('can', 38863), ('will', 36865), ('at', 35548), ('an', 31392), ('money', 31003), ('so', 29980), ('$', 29096), ('would', 28750), ('from', 28582), ('more', 27378)]


In [ ]:
print(len(train_set), len(test_set))

6315 333


Save the following:
1. Save train_set, and test_set
2. Save qid_to_text, docid_to_text
3. save text_to_docid

In [ ]:
# define dictionary
save_pickle(WR_INTERIM_DATA + "train_set.pickle", train_set)
save_pickle(WR_INTERIM_DATA + "test_set.pickle", test_set)
save_pickle(WR_INTERIM_DATA + "qid_to_text.pickle", qid_to_text)
save_pickle(WR_INTERIM_DATA + "docid_to_text.pickle", docid_to_text)
save_pickle(WR_INTERIM_DATA + "text_to_docid.pickle", text_to_docid)